In [1]:
%matplotlib osx
%load_ext autoreload
%autoreload 2

In [5]:
from lib import jsoncorpus, datastuff, analysis
import gensim
import traceback
import numpy as np
import matplotlib.pyplot as plt
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.cross_validation
import sklearn.ensemble
import sklearn.svm
import sklearn.decomposition
import sklearn.cluster
import sklearn.manifold
from lib.scikitComponents import *
import itertools
import collections
import sklearn.metrics

# LDA

## Data preprocessing

In [6]:
# Load in the previously created corpus and dictionary of scraped sites
dictionary, corpus, meta_corpus, dmoz_data = jsoncorpus.load_or_create('docs/sites.jl', stemmed=True, prefix="stemmed_")

Done.


In [57]:
# Link topics to URLs
meta = list(zip(dmoz_data['urls'], dmoz_data['dmoz_categories']))
# Build array of class labels
classes, top_categories, dmoz_encoder = datastuff.encode_dmoz_categories(dmoz_data['dmoz_categories'])
# Set up datasets
X = np.array(meta_corpus)
y = classes

In [58]:
#clustering = 
#clustering = sklearn.manifold.TSNE(init='pca')
#clustering = sklearn.cluster.MeanShift()
pipe = sklearn.pipeline.Pipeline([
    ('tfidf', TFIDFModel(id2word=dictionary)),
    ('matrixbuilder', TopicMatrixBuilder(len(dictionary.items()), keep_all=True)),
    ('kmeans', sklearn.cluster.KMeans(n_clusters = 20)),
#    ('tSNE-pca', sklearn.manifold.TSNE(init='pca')),
    ('tSNE', sklearn.manifold.TSNE()),
])
# Clustering!
transformed = pipe.fit_transform(X)
#pipe.fit(X)

Pipeline(steps=[('tfidf', TFIDFModel(id2word=None)), ('matrixbuilder', TopicMatrixBuilder(keep_all=True, num_topics=13021, topic_min_members=0)), ('kmeans', KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=20, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)), ('tSNE', TSNE(early_exaggeration=4.0, init='random', learning_rate=1000.0,
   metric='euclidean', n_components=2, n_iter=1000, perplexity=30.0,
   random_state=None, verbose=0))])

In [66]:
#transformed = pipe.transform(X)
predicted = sklearn.pipeline.Pipeline(pipe.steps[:-1]).predict(X)

In [67]:
dmoz_encoder.classes_

array([u'arts', u'business', u'computers', u'games', u'health', u'home',
       u'kids and teens', u'news', u'recreation', u'reference',
       u'regional', u'science', u'shopping', u'society', u'sports'], 
      dtype='<U14')

In [73]:
fig = plt.figure()
plt.scatter(transformed[:, 0], transformed[:, 1], c = predicted)

In [75]:
cols = ['c','g','b', 'r','y','k','m', 'c','g','b','r','y','k','m', 'c']
markers = ['o']*7 + ['x']*7 + ['^']
plt.hold(True)
for x in range(max(y)+1):    
    plt.scatter(transformed[y==x, 0], transformed[y==x, 1], c = cols[x], marker=markers[x])

plt.legend(dmoz_encoder.classes_, loc='upper left')
plt.hold(False)

In [37]:
#for i, x in enumerate(meta):
#    if predicted[i] == 0:
#        print meta[i]